****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.


## imports for Python, Pandas, json

In [4]:
import pandas as pd
import json
from pandas.io.json import json_normalize

#pd.set_option('display.height', 1000)
#pd.set_option('display.max_rows', 1000)
#pd.set_option('display.max_columns', 1000)
#pd.set_option('display.width', 1000)
#pd.set_option('display.max_colwidth', -1)

sample_json_df = pd.read_json('data/world_bank_projects.json')
sample_json_df.head(2)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
# 1. Find the 10 countries with most projects
****

##### Select country code and country short name from json dataframe. Group by country code and take the count. Sort resulting dataframe by counts descending. Select top 10 values from resulting dataframe.


In [8]:
top10CountriesProjCt = sample_json_df[['countrycode', 'countryshortname']] \
                             .groupby(['countrycode', 'countryshortname'])['countrycode'] \
                             .count() \
                             .reset_index(name='counts') \
                             .sort_values(['counts'], ascending=False) \
                             .head(10)
top10CountriesProjCt

,countrycode,countryshortname,counts
45,ID,Indonesia,19
28,CN,China,19
110,VN,Vietnam,17
46,IN,India,16
88,RY,"Yemen, Republic of",13
60,MA,Morocco,12
15,BD,Bangladesh,12
78,NP,Nepal,12
1,3A,Africa,11
73,MZ,Mozambique,11


### Indonesia, China, Vietnam, India, Yemen, Morocco, Bangladesh, Nepal, Africa, Mozambique are top 10 countries with most projects.

#### Suppose the question is - Find top most contry with heighest number of project? In that case there are two countries with 19 projects. I feel choose top countries based on their ranking is a better idea.


In [10]:
# Top 10 countries based on dense ranking
top10CountriesProjCtDRank = sample_json_df[['countrycode', 'countryshortname']] \
                             .groupby(['countrycode', 'countryshortname'])['countrycode'] \
                             .count() \
                             .reset_index(name='counts') \
                             .sort_values(['counts'], ascending=False) 

top10CountriesProjCtDRank['Rank'] = top10CountriesProjCtDRank.counts.rank(method='dense', ascending =False).astype(int)
top10CountriesProjCtDRank[top10CountriesProjCtDRank['Rank'] < 11]

,countrycode,countryshortname,counts,Rank
45,ID,Indonesia,19,1
28,CN,China,19,1
110,VN,Vietnam,17,2
46,IN,India,16,3
88,RY,"Yemen, Republic of",13,4
60,MA,Morocco,12,5
15,BD,Bangladesh,12,5
78,NP,Nepal,12,5
1,3A,Africa,11,6
73,MZ,Mozambique,11,6


 #### Indonesia, China, Vietnam, India, Yemen, Morocco, Bangladesh, Nepal, Africa, Mozambique, Pakistan, Burkina Faso, Brazil, Armenia, Tanzania, Tajikistan, Jordan, Nigeria, Kyrgyz Republic, Lao People''s Democratic Republic, Kenya, West Bank and Gaza, Nicaragua, Afghanistan, Congo, Peru are top 10 ranked countries with more number projects.

****
# 2. Find the top 10 major project themes (using column 'mjtheme_namecode')
****

##### Read json file using json.load. It becomes easy to normalize nested jason when you read it as list of strings. Normalize mjtheme_namecode column. Count total number of projects per theme code. Sort the result by counts descending. Create a look up table for themes. Then find top 10 project theme names. 

## imports for Python, Pandas, json, numpy

In [32]:
import pandas as pd
import json
import numpy as np
from pandas.io.json import json_normalize
json_data=json.load((open('data/world_bank_projects.json')))
json_data[0]

{'_id': {'$oid': '52b213b38594d8a2be17c780'},
 'approvalfy': 1999,
 'board_approval_month': 'November',
 'boardapprovaldate': '2013-11-12T00:00:00Z',
 'borrower': 'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
 'closingdate': '2018-07-07T00:00:00Z',
 'country_namecode': 'Federal Democratic Republic of Ethiopia!$!ET',
 'countrycode': 'ET',
 'countryname': 'Federal Democratic Republic of Ethiopia',
 'countryshortname': 'Ethiopia',
 'docty': 'Project Information Document,Indigenous Peoples Plan,Project Information Document',
 'envassesmentcategorycode': 'C',
 'grantamt': 0,
 'ibrdcommamt': 0,
 'id': 'P129828',
 'idacommamt': 130000000,
 'impagency': 'MINISTRY OF EDUCATION',
 'lendinginstr': 'Investment Project Financing',
 'lendinginstrtype': 'IN',
 'lendprojectcost': 550000000,
 'majorsector_percent': [{'Name': 'Education', 'Percent': 46},
  {'Name': 'Education', 'Percent': 26},
  {'Name': 'Public Administration, Law, and Justice', 'Percent': 16},
  {'Name': 'Education', 'Percent': 12}],
 'm

In [42]:
jsonNorm = json_normalize(json_data, 'mjtheme_namecode', ['countrycode', 'project_name'])
jsonNorm['code'] = pd.to_numeric(jsonNorm['code'])
jsonNorm.head(2)

,code,name,countrycode,project_name
0,8,Human development,ET,Ethiopia General Education Quality Improvement...
1,11,,ET,Ethiopia General Education Quality Improvement...


In [60]:
top10ProjThemes = jsonNorm[['code']] \
                             .groupby(['code'])['code'] \
                             .count() \
                             .reset_index(name='counts') \
                             .sort_values(['counts'], ascending=False) 
top10ProjThemes.head(10)

,code,counts
10,11,250
9,10,216
7,8,210
1,2,199
5,6,168
3,4,146
6,7,130
4,5,77
8,9,50
0,1,38


In [61]:
json_data_lookup = json_normalize(json_data, 'mjtheme_namecode').drop_duplicates().sort_values(by=['code'])
json_data_lookup.head(2)

,code,name
2,1,Economic management
212,1,


In [62]:
json_data_lookup['code'] = pd.to_numeric(json_data_lookup['code'])
json_data_lookup = json_data_lookup.groupby('code', sort=True)['name'].max().to_frame()
json_data_lookup.reset_index(inplace=True)
json_data_lookup

,code,name
0,1,Economic management
1,2,Public sector governance
2,3,Rule of law
3,4,Financial and private sector development
4,5,Trade and integration
5,6,Social protection and risk management
6,7,Social dev/gender/inclusion
7,8,Human development
8,9,Urban development
9,10,Rural development


In [63]:
if (top10ProjThemes['code'].dtype == json_data_lookup['code'].dtype):
    print('You can join datafrmes')
else:
    print('Convert column to correct data type')

You can join datafrmes


In [70]:
result = pd.merge(top10ProjThemes, json_data_lookup, on=['code'])
result

,name
0,Environment and natural resources management
1,Rural development
2,Human development
3,Public sector governance
4,Social protection and risk management
5,Financial and private sector development
6,Social dev/gender/inclusion
7,Trade and integration
8,Urban development
9,Economic management


#### Environment and natural resources management, Rural development, Human development, Public sector governance, Social protection and risk management, Financial and private sector development, Social dev/gender/inclusion, Trade and integration, Urban development, Economic management, Rule of law are 10 major project themes

****
# 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.
****

##### Replace missing values in results dataframe with NaN. Join fillMissingNaN dataframe with theme lookup table. it will create name_x and name_y. Create a new column name and merge name_x and name_y by taking max between the two. Drop name_x and name_y.

In [74]:
fillMissingNaN = jsonNorm.replace(r'^\s*$', np.nan, regex=True)
merged = pd.merge(left=jsonNorm,right=json_data_lookup, how='inner', on='code')
merged['name'] = np.max(merged[['name_x', 'name_y']], axis=1)
merged = merged.drop(labels=['name_x', 'name_y'], axis=1)
merged

,code,countrycode,project_name,name
0,8,ET,Ethiopia General Education Quality Improvement...,Human development
1,8,CN,China Renewable Energy Scale-Up Program Phase II,Human development
2,8,MG,Emergency Support to Education For all Project,Human development
3,8,KH,Additional Financing for the Second Health Sec...,Human development
4,8,KH,Additional Financing for the Second Health Sec...,Human development
5,8,KH,Additional Financing for the Second Health Sec...,Human development
6,8,TJ,Tajikistan JSDF Nutrition Grant Scale Up,Human development
7,8,TJ,Tajikistan JSDF Nutrition Grant Scale Up,Human development
8,8,LA,Lao PDR Ninth Poverty Reduction Support Operation,Human development
9,8,AO,Angola Learning for All Project,Human development
